In [1]:
#!pip install -U aeon
#!pip install aeon[all_extras]
import warnings
import pandas as pd
from matplotlib import pyplot as plt
from aeon.visualisation import plot_series
from all_functions import *
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb 
from sklearn.model_selection import GridSearchCV
from aeon.transformations.detrend import STLTransformer
import ast
from sklearn.metrics import mean_absolute_percentage_error as mape
from sklearn.metrics import mean_squared_error, make_scorer
import os
from sklearn.linear_model import Ridge, RidgeCV
warnings.filterwarnings("ignore")
%matplotlib inline
def convert_to_list(series_str):
    return eval(series_str)

def read_tsf(file_path):
    with open(file_path, 'r', encoding='ISO-8859-1') as f:
        lines = f.readlines()

    metadata = []
    series = []

    for line in lines:
        if line.startswith('@') or line.startswith('#'):
            metadata.append(line.strip())
        else:
            series.append(line.strip())

    formatted_data = []

    # Processar cada linha de série
    for entry in series:
        dataset_name, start_date, series_values = entry.split(':', 2)
        start_date = start_date.strip()
        series_list = [float(value) for value in series_values.split(',')]
        
        # Adicionar aos dados formatados
        formatted_data.append({
            'dataset': dataset_name,
            'start_date': start_date,
            'series': series_list
        })
        
    series_data = pd.DataFrame(formatted_data)


    return metadata, series_data

/home/anp/anaconda3/lib/python3.11/site-packages/aeon/base/__init__.py:24: FutureWarning: The aeon package will soon be releasing v1.0.0 with the removal of legacy modules and interfaces such as BaseTransformer and BaseForecaster. This will contain breaking changes. See aeon-toolkit.org for more information. Set aeon.AEON_DEPRECATION_WARNING or the AEON_DEPRECATION_WARNING environmental variable to 'False' to disable this warning.
  warnings.warn(


In [34]:
import numpy as np
import pandas as pd

# Função para calcular os valores de SMAPE
def calculate_smape(forecasts, test_set):
    smape = 2 * np.abs(forecasts - test_set) / (np.abs(forecasts) + np.abs(test_set))
    smape_per_series = np.nanmean(smape, axis=1)  # Média por série
    return smape_per_series

# Função para calcular os valores de mSMAPE
def calculate_msmape(forecasts, test_set):
    epsilon = 0.1
    comparator = np.full(test_set.shape, 0.5 + epsilon)
    sum_values = np.maximum(comparator, (np.abs(forecasts) + np.abs(test_set) + epsilon))
    smape = 2 * np.abs(forecasts - test_set) / sum_values
    msmape_per_series = np.nanmean(smape, axis=1)
    return msmape_per_series

# Função para calcular os valores de MASE
def calculate_mase(forecasts, test_set, training_set, seasonality):
    mase_per_series = []
    
    for k in range(forecasts.shape[0]):
        te = test_set[k, ~np.isnan(test_set[k, :])]
        tr = training_set[k][~np.isnan(training_set[k])]
        f = forecasts[k, ~np.isnan(forecasts[k, :])]
        
        # Cálculo de MASE
        mase = MASE(te, f, np.mean(np.abs(np.diff(tr, n=1, axis=0, prepend=tr[0]))))
        
        if np.isnan(mase):
            mase = MASE(te, f, np.mean(np.abs(np.diff(tr, n=1, axis=0, prepend=tr[0]))))
        
        mase_per_series.append(mase)

    return np.array(mase_per_series)

# Função para calcular os valores de MAE
def calculate_mae(forecasts, test_set):
    mae = np.abs(forecasts - test_set)
    mae_per_series = np.nanmean(mae, axis=1)
    return mae_per_series

# Função para calcular os valores de RMSE
def calculate_rmse(forecasts, test_set):
    squared_errors = (forecasts - test_set) ** 2
    rmse_per_series = np.sqrt(np.nanmean(squared_errors, axis=1))
    return rmse_per_series

# Função para fornecer um resumo das métricas de erro
def calculate_errors(forecasts, test_set, training_set, seasonality, output_file_name):
    smape_per_series = calculate_smape(forecasts, test_set)
    msmape_per_series = calculate_msmape(forecasts, test_set)
    mase_per_series = calculate_mase(forecasts, test_set, training_set, seasonality)
    mae_per_series = calculate_mae(forecasts, test_set)
    rmse_per_series = calculate_rmse(forecasts, test_set)

    metrics = {
        "Mean SMAPE": np.nanmean(smape_per_series),
        "Median SMAPE": np.nanmedian(smape_per_series),
        "Mean mSMAPE": np.nanmean(msmape_per_series),
        "Median mSMAPE": np.nanmedian(msmape_per_series),
        "Mean MASE": np.nanmean(mase_per_series),
        "Median MASE": np.nanmedian(mase_per_series),
        "Mean MAE": np.nanmean(mae_per_series),
        "Median MAE": np.nanmedian(mae_per_series),
        "Mean RMSE": np.nanmean(rmse_per_series),
        "Median RMSE": np.nanmedian(rmse_per_series),
    }

    for key, value in metrics.items():
        print(f"{key}: {value}")

    # Escrevendo as métricas em arquivos
    np.savetxt(f"{output_file_name}_smape.txt", smape_per_series, delimiter=",")
    np.savetxt(f"{output_file_name}_msmape.txt", msmape_per_series, delimiter=",")
    np.savetxt(f"{output_file_name}_mase.txt", mase_per_series, delimiter=",")
    np.savetxt(f"{output_file_name}_mae.txt", mae_per_series, delimiter=",")
    np.savetxt(f"{output_file_name}_rmse.txt", rmse_per_series, delimiter=",")
    
    with open(f"{output_file_name}.txt", "w") as f:
        for key, value in metrics.items():
            f.write(f"{key}: {value}\n")

# Função auxiliar para calcular MASE
def MASE(actual, forecast, training_mean):
    return np.mean(np.abs(actual - forecast)) / training_mean if training_mean != 0 else np.nan


def morlet_wavelet(f, t, w=5):
    """Gera a wavelet de Morlet."""
    if f == 0:
        return np.zeros_like(t)
    return np.exp(1j * 2 * np.pi * f * t) * np.exp(-t ** 2 / (2 * (w / (2 * np.pi * f)) ** 2))

def apply_kernel(X, num_kernels, lengths, dilations, paddings, biases):
    X = np.array(X)
    input_length = len(X)

    features = np.zeros((input_length, num_kernels))

    for k in range(num_kernels):
        length = lengths[k]
        padding = paddings[k]
        bias = biases[k]
        dilation = dilations[k]

        end = (input_length + padding) - ((length - 1) * dilation)

        for i in range(-padding, end):
            _sum = bias
            index = i

            for j in range(length):
                wavelet_index = index + j * dilation
                if 0 <= wavelet_index < input_length:
                    frequency = 1 / (length / 2)
                    wavelet_value = morlet_wavelet(frequency, j / length)
                    _sum += wavelet_value.real * X[wavelet_index]
            
            if i + padding >= 0:
                features[i + padding, k] = _sum

    return features, np.sum(features > 0)

def features_target(series, window):
    data = []

    num_kernels = 1
    # lengths = np.array([3, 5, 7])
    # dilations = np.array([1, 2, 1])
    # paddings = np.array([1, 0, 2])
    # biases = np.random.uniform(-1, 1, size=num_kernels)

    weights, lengths, biases, dilations, paddings = generate_kernels(len(series), num_kernels)
    
    for i in range(len(series) - window):
      example = np.array(series[i:i+window])
      target_value = series[i+window]
      features, ppv_count = apply_kernel(example, num_kernels, lengths, dilations, paddings, biases)
      data.append(np.concatenate((features.flatten(), [target_value])))

      df = pd.DataFrame(data)
    return df


def generate_kernels(input_length, num_kernels):

    candidate_lengths = np.array((7, 9, 11), dtype = np.int32)
    lengths = np.random.choice(candidate_lengths, num_kernels)

    weights = np.zeros(lengths.sum(), dtype = np.float64)
    biases = np.zeros(num_kernels, dtype = np.float64)
    dilations = np.zeros(num_kernels, dtype = np.int32)
    paddings = np.zeros(num_kernels, dtype = np.int32)

    a1 = 0

    for i in range(num_kernels):
        _length = lengths[i]

        _weights = np.random.normal(0, 1, _length)

        b1 = a1 + _length
        weights[a1:b1] = _weights - _weights.mean()

        biases[i] = np.random.uniform(-1, 1)
        dilation = 2 ** np.random.uniform(0, np.log2((input_length - 1) / (_length - 1)))
        dilation = np.int32(dilation)
        dilations[i] = dilation

        padding = ((_length - 1) * dilation) // 2 if np.random.randint(2) == 1 else 0
        paddings[i] = padding

        a1 = b1

    return weights, lengths, biases, dilations, paddings

In [35]:
def ensure_even(n):
    return n + (n % 2)


def recursive_rocket2(X_test, model, feats, horizon):
  # example é composto pelas últimas observações vistas
  # na prática, é o primeiro exemplo do conjunto de teste
  num_kernels = 1
  # lengths = np.array([3, 5, 7])
  # dilations = np.array([1, 2, 1])
  # paddings = np.array([1, 0, 2])
  # biases = np.random.uniform(-1, 1, size=num_kernels)
  weights, lengths, biases, dilations, paddings = generate_kernels(len(feats), num_kernels)
  example = X_test.iloc[0].values.reshape(1,-1)
  preds = []
  for i in range(horizon):
    pred = model.predict(example)[0]
    preds.append(pred)
    feats.append(pred)
    feats = feats[1:]

   # example = example[:,2:]

    # print(example)
    transformed_feats, _ = apply_kernel(feats, num_kernels, lengths, dilations, paddings, biases)
    # example = np.append(example, feats)
    # example = example.reshape(1,-1)
    example = transformed_feats.flatten()
    example = example.reshape(1,-1)
  return preds

In [14]:
metadata, series_data = read_tsf('../m4/m4_monthly_dataset.tsf')
for meta in metadata:
    if '@horizon' in meta:
        h = meta[9:]

int(h)

18

In [36]:
metadata, series_data = read_tsf('../m4/m4_weekly_dataset.tsf')
horizon = 13
representation = "DWT"
level = 7
wavelet = "bior2.2"
transform = "normal"
window = 12
# window = 90

# series = pd.read_csv('../m4/m4_monthly_dataset.tsf', sep='\t', encoding='ISO-8859-1')
smapes = []
rmses = []
for index in range(len(series_data)):
    try:
        series = series_data.loc[index]['series']

        train, test = train_test_stats(pd.Series(series), horizon)
        train_tf = transform_train(train, transform)
        data = features_target(pd.concat([train_tf, pd.Series([0] * horizon, index=test.index)]), window)
        # data = rolling_window(pd.concat([train_tf, pd.Series([0] * horizon, index=test.index)]), window)
        X_train, X_test, y_train, _ = train_test_split(data, horizon)
        # rg = RidgeCV(alphas=np.logspace(-3, 3, 10))
        rg = RandomForestRegressor(random_state=42)
        rg.fit(X_train, y_train)

        predictions = recursive_rocket2(X_test, rg, train_tf[-window:].tolist(), horizon)

        # predictions = recursive_multistep_forecasting(X_test, rg, horizon)
        preds = pd.Series(predictions, index=test.index)
        preds_real = reverse_regressors(train, preds, format=transform)
        mape_result = mape(test, preds_real)

        preds_real_array = np.array(preds_real)
        preds_real_reshaped = preds_real_array.reshape(1, -1)
        test_reshaped = test.values.reshape(1, -1)
        smape_result = calculate_smape(preds_real_reshaped, test_reshaped)
        rmse_result = calculate_rmse(preds_real_reshaped, test_reshaped)
        # pocid_result = pocid(test, preds_real)
        print(index)
        rmses.append(rmse_result)
        smapes.append(smape_result)
        print(np.mean(rmses))
    except Exception as e:
        print(f'Index: {index} | {e}')
        print(len(series))
        continue
    # print(pocid_result)

0
1494.8910740127774
1
817.8661397577401
2
680.8954137470688
3
714.132136595855
4
658.6774947138101
5
609.3420837100128
6
768.1889408436191
7
1024.0984424778223
8
1187.911312869587
Index: 9 | Input X contains NaN.
RandomForestRegressor does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values
947
10
1074.0345505552377
11
978.7130392238737
12
919.4363892484339
13
862.3273611757321
14
878.9420658958499
15
926.9904190964622
16
889.5851604315649
17
853.98093

In [37]:
np.mean(smapes)

0.11680836857304337

In [38]:
np.mean(rmses)

527.3371636072958